# Manipulating databases

* Produce PQ db extension to GAP-18
* initiate active learning from GAP

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from ase.io.extxyz import read_xyz
import numpy as np
from ase.atoms import Atoms
from ase.io.formats import index2range
from ase.io.cfg import read_cfg
from io import StringIO, UnsupportedOperation
from mtp import *
import os
from matplotlib import pyplot as plt
from quippy.potential import Potential
import pickle
from Ge_analysis import *
from Ge_calculation import *
import matplotlib.pyplot as plt
from matscipy.rings import ring_statistics
from datetime import datetime
import pymatgen.ext.matproj as mp
import pymatgen.io.ase as pase
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from numpy.random import normal
from ase.io.extxyz import write_xyz
from ase.io.lammpsdata import write_lammps_data

In [3]:
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = [ 'Helvetica', 'Arial', 'FreeSans', 'DejaVu Sans' ]

plt.rcParams['axes.labelsize'] = 18
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15

plt.rcParams['legend.fontsize'] = 14
plt.rcParams['legend.title_fontsize'] = 14
plt.rcParams['legend.fancybox'] = False
plt.rcParams['legend.edgecolor'] = 'k'
plt.rcParams['legend.borderaxespad'] = 1.5

plt.rcParams['lines.linewidth'] = 3.0
plt.rcParams['axes.linewidth'] = 1.5

plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['xtick.major.size'] = 10
plt.rcParams['xtick.major.width'] = 1.0
plt.rcParams['xtick.minor.size'] = 5
plt.rcParams['xtick.minor.width'] = 1.0

plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['ytick.major.size'] = 10
plt.rcParams['ytick.major.width'] = 1.0
plt.rcParams['ytick.minor.size'] = 5
plt.rcParams['ytick.minor.width'] = 1.0

plt.rcParams["figure.autolayout"] = False

# Labelling

In [4]:
GAP_18_dir = '/u/vld/hert5155/jup_remote/Ge_od/Si_GAP_training/libAtoms-silicon-testing-framework-fc252cb/models/GAP'
os.chdir('/u/vld/hert5155/jup_remote/Ge_od/GAP_to_MTP')
GAP_18_pot = Potential(param_filename=GAP_18_dir+'/gp_iter6_sparse9k.xml')

/u/vld/hert5155/ase/ase/calculators/calculator.py:507: FutureWarning: The keyword "ignore_bad_restart_file" is deprecated and will be removed in a future version of ASE.  Passing more than one positional argument to Calculator is also deprecated and will stop functioning in the future.  Please pass arguments by keyword (key=value) except optionally the "restart" keyword.
  warnings.warn(FutureWarning(


In [29]:
with open(GAP_18_dir + '/gp_iter6_sparse9k.xml.xyz', 'r') as f:
    GAP_18_db = list(read_xyz(f, index=slice(0,None)))

In [ ]:
with open('/home/joe/Documents/Ge_od/Potentials/Si_myDB_reduction_testing/Si_myDB_64_125_216_train.xyz', 'r') as f:
    myDB = list(read_xyz(f, index=slice(0,None)))

In [ ]:
db = myDB
# db = GAP_18_db
for i, val in enumerate(db):

    GAP_18_pot.calculate(val, properties=['energies', 'forces', 'stress'])
    val.info['gap18_energy'] = GAP_18_pot.results['energy']
    val.info['gap18_virial'] = GAP_18_pot.results['virial']
    val.arrays['gap18_force'] = GAP_18_pot.results['forces']

In [ ]:
GAP_18_pot.calculate(GAP_18_db[0], properties=['energies', 'forces', 'stress'])
GAP_18_pot.results['virial']

In [41]:
mtp_stress_test = MTP('/u/vld/hert5155/jup_remote/Ge_od/GAP_to_MTP/MTPs/MTPs/u16_m1.5_fw0.1_GAP18db.mtp',
                     mtp_command='/u/vld/hert5155/mlip-2/bin/mlp')

In [ ]:
# with open('GAP_18_db_GAPlabelled_MTP4.pickle', 'wb') as f:
#     pickle.dump(GAP_18_db, f)

In [5]:
with open('GAP_18_db_GAPlabelled.pickle', 'rb') as f:
    GAP_18_db = pickle.load(f)

In [ ]:
# with open('mydb_GAPlabelled_MTP4.pickle', 'wb') as f:
#     pickle.dump(myDB, f)

In [ ]:
with open('mydb_GAPlabelled_MTP4.pickle', 'rb') as f:
    myDB = pickle.load(f)

In [ ]:
# with open('GAP_18_db_GAPlabelled.cfg', 'w') as f:
#     write_cfg_db(f, GAP_18_db, energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_virial')

In [ ]:
# with open('GAP_18_db_DFTlabelled.cfg', 'w') as f:
#     write_cfg_db(f, GAP_18_db, energy_name='dft_energy', force_name='dft_force', virial_name='dft_virial')

## Writing all melt-quench data to DB for fitting


In [ ]:
sb = '/home/joe/Documents/Ge_od/Structure_databases'
mq_dirs = [os.path.join(sb, i) for i in ['run_64_atoms', 'run_125_atoms', 'run_216_atoms']]

mq_db = []

# for i in mq_dirs:
#     for j in os.listdir(i):
#         if 'run_' in j and 'castep' not in j:
#             for k in os.listdir(os.path.join(i, j, 'NPT', )):
#                 at = read_cfg(os.path.join(i,j,'NPT',k))
#                 at.info['rundir'] = j
#                 mq_db.append(at)
# l = len(mq_db)
# for i, val in enumerate(mq_db):
#
#     GAP_18_pot.calculate(val, properties=['energies', 'forces', 'stress'])
#     val.info['gap18_energy'] = GAP_18_pot.results['energy']
#     val.info['gap18_virial'] = GAP_18_pot.results['virial']
#     val.arrays['gap18_force'] = GAP_18_pot.results['forces']
#     if (i//l * 100) % 10 == 0:
#         print('{}% done'.format(i//l *100), datetime.now().time())

# with open('mq_db_GAP-18labelled.pickle', 'wb') as f:
#     pickle.dump(mq_db, f)

In [57]:
with open('/home/joe/Documents/Ge_od/GAP_to_MTP/mqdb_GAP18labelled/mq_db_GAP-18labelled_3.pickle', 'rb') as f:
    mq_db_G18lab = pickle.load(f)


In [58]:
# need to modify the stresses before writing them
for i in range(len(mq_db_G18lab)):
    v = -mq_db_G18lab[i].info['gap18_virial']*mq_db_G18lab[i].get_volume()
    mq_db_G18lab[i].info['gap18_virial'] = v


In [59]:
# split into 80:20 test:training with entirely separate runs
rundirs = [int(i.info['rundir'][4:]) for i in mq_db_G18lab]
rundirs = np.array(rundirs)

In [60]:
# keep back run numbers 8-27
v_args = np.array([[]]); t_args = np.array([[]])
for i in range(8,28):
    v_args = np.concatenate((v_args, np.where(rundirs==i)), axis=1)

for i in range(8):
    t_args = np.concatenate((t_args, np.where(rundirs==i)), axis=1)
for i in range(28,101):
    t_args = np.concatenate((t_args, np.where(rundirs==i)), axis=1)

In [61]:
mq_db_G18lab_train = [mq_db_G18lab[int(i)] for i in t_args[0]]
mq_db_G18lab_val = [mq_db_G18lab[int(i)] for i in v_args[0]]

In [62]:
# run_97 etc. didn't finish, so not exactly 80% but pretty close
len(mq_db_G18lab_train)/len(mq_db_G18lab_val)


len(mq_db_G18lab_train),len(mq_db_G18lab_val)

(22810, 5784)

In [ ]:
# with open('mqdb_GAP18labelled/mqdb_GAP18labelled_train.cfg', 'w') as f:
#     write_cfg_db(f, mq_db_G18lab_train, energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_virial')

In [ ]:
# with open('mqdb_GAP18labelled/mqdb_GAP18labelled_validate.cfg', 'w') as f:
#     write_cfg_db(f, mq_db_G18lab_val, energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_virial')

In [ ]:
GAP18_mqdb_train = GAP_18_db + mq_db_G18lab_train

In [ ]:

GAP18_mqdb_train = GAP_18_db + mq_db_G18lab_train

In [66]:
#sum([len(i) for i in mq_db_G18lab_train])
sum([len(i) for i in GAP_18_db])


171815

In [ ]:
# with open('mqdb_GAP18labelled/GAP18_mqdb_GAP18labelled_train.cfg', 'w') as f:
#     write_cfg_db(f, GAP18_mqdb_train, energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_virial')


## Generate GAP_labelled crystal structures from the MP
Free us from dependence on the GAP-18 database

In [ ]:
import pymatgen.ext.matproj as mp
import pymatgen.io.ase as pase
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

In [ ]:
with mp.MPRester("v0JVgKLlpTXKEhiKeDIC") as m:
    results = m.query(criteria={"pretty_formula":'Si'},
                            properties=["exp.tags", "icsd_ids", "spacegroup",
                                        "task_id", "density", "final_structure",
                                        "e_above_hull", "energy", "energy_per_atom"])

In [ ]:
mp_xtals = [pase.AseAtomsAdaptor().get_atoms(
            SpacegroupAnalyzer(i['final_structure']).get_conventional_standard_structure())
            for i in results]

In [ ]:
def get_supercell_factor(at, target_size=64):
    f = np.ceil((target_size/len(at))**(1/3))
    #print('supercell factor: %f leads to %f atoms' % (f, len(at)*f**3))
    return f

In [ ]:
# ncopies = 50
ncopies = 10
supers = [[] for i in results]
for i, val in enumerate(mp_xtals):
    for j in range(ncopies):
        s = build.supercells.make_supercell(
                val, get_supercell_factor(val)*np.identity(3))
        s.rattle(0.01) # randomise the positions slightly
        s.set_cell(s.get_cell_lengths_and_angles() * np.concatenate([normal(1, 0.03, (3)), normal(1, 0.01, (3))]),
                 scale_atoms=True) # randomise lattice vectors
        s.info['config_type'] = results[i]['task_id']
        supers[i].append(s)

In [ ]:
sum([len(i) for i in flatten(supers)]), sum([len(i) for i in mq_db_G18lab_train])

In [ ]:
# need to label with GAP-18 now
with open('mp_mq_db_GAP18labelled/mp_db_train_test.xyz', 'w') as f:
    write_xyz(f, flatten(supers[:5]))

In [ ]:
# with open('mp_mq_db_GAP18labelled/mp_db_train_test.cfg', 'w') as f:
#     write_cfg_db(f, mp_db_GAP18labelled_train[:5], energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_virial')

In [ ]:
with open('mp_mq_db_GAP18labelled/mp_db_validate.xyz', 'w') as f:
    write_xyz(f, flatten(supers))

In [ ]:
with open('mp_mq_db_GAP18labelled/mp_db_GAP18labelled_train.xyz', 'r') as f:
    mp_db_GAP18labelled_train = list(read_xyz(f, index=slice(0,None)))

In [ ]:
with open('mp_mq_db_GAP18labelled/mp_db_GAP18labelled_validate.xyz', 'r') as f:
    mp_db_GAP18labelled_val = list(read_xyz(f, index=slice(0,None)))

In [ ]:
for i in mp_db_GAP18labelled_train:
    i.info['gap18_energy'] = i.info.pop('energy')
    i.info['gap18_virial'] = i.info.pop('virial')
    i.arrays['gap18_force'] = i.arrays.pop('force')


In [ ]:
mp_mq_db_GAP18labelled_train = mq_db_G18lab_train + mp_db_GAP18labelled_train

In [ ]:
with open('mp_mq_db_GAP18labelled/mp_mq_db_GAP18labelled_train.cfg', 'w') as f:
    write_cfg_db(f, mp_mq_db_GAP18labelled_train, energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_virial')

# Big melt quench development


In [34]:
#amo_1k = read_cfg('/home/joe/Documents/Ge_od/GAP_to_MTP/pressure_quench_devel/dump_npt_Si_mtp.240000.cfg')
amo_1k_litGAP = read_cfg('/home/joe/Documents/Ge_od/GAP_to_MTP/MTP_mds/GAPmd_run_1000_lit/NPT/dump_npt_Si_gap.240000.cfg')

In [37]:
write_lammps_data('/home/joe/Documents/Ge_od/GAP_to_MTP/pressure_quench/amo_1k_litGAP.data', amo_1k_litGAP)

In [27]:
with open('/home/joe/Documents/Ge_od/GAP_to_MTP/MTP_mds/u16_m1.5_run10k/NPT/dump_npt_Si_mtp.240000.cfg', 'r') as f:
    amo_10k = read_cfg(f)

In [48]:
def label_db(db, pot, pot_name, e_name='energy', f_name='force', v_name='virial'):

    for i, val in enumerate(db):
        pot.calculate(val, properties=['energies', 'forces', 'stress'])
        val.info['{}_{}'.format(pot_name, e_name)] = pot.results['energy']
        val.info['{}_{}'.format(pot_name, v_name)] = pot.extra_results['config']['virial']
        val.arrays['{}_{}'.format(pot_name, f_name)] = pot.results['forces']


In [48]:
with open('PQ_GAP18_GAP18labelled/aSi_100k_relaxed_amorphous_labelled.xyz', 'r') as f:
    amo_100k = list(read_xyz(f))
# label_db(amo_100k, GAP_18_pot, 'gap18')


In [54]:
amo_100k[0].info['gap18_energy'] = amo_100k[0].info['energy']
amo_100k[0].info['gap18_virial'] = amo_100k[0].info['virial']
amo_100k[0].arrays['gap18_force'] = amo_100k[0].arrays['force']

In [55]:
PQ100k_GAP18_db_GAP18labelled = GAP_18_db + amo_100k

In [56]:
with open('PQ_GAP18_GAP18labelled/PQ100k_GAP18_db_GAP18labelled.cfg', 'w') as f:
    write_cfg_db(f, PQ100k_GAP18_db_GAP18labelled,
                 energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_virial')


In [ ]:
write_lammps_data('/home/joe/Documents/Ge_od/GAP_to_MTP/pressure_quench/dump_npt_Si_mtp.240000.data', amo_1k)

In [42]:
write_lammps_data('/home/joe/Documents/Ge_od/GAP_to_MTP/pressure_quench/amo_100k.data', amo_100k)

In [29]:
write_lammps_data('/home/joe/Documents/Ge_od/GAP_to_MTP/pressure_quench/amo_10k.data', amo_10k)

## Write 1k pressure quench to DB for additional fitting

In [12]:
GAP18_PQ = MD_run('/u/vld/hert5155/jup_remote/Ge_od/GAP_to_MTP/GAP-18_ref/GAP18_PQ_1k/',
                  label='GAP18_PQ_1k', read_dat=True)

In [63]:
GAP18_PQ.df['Configs'][1000].info

{'file': '/u/vld/hert5155/jup_remote/Ge_od/GAP_to_MTP/GAP-18_ref/GAP18_PQ_1k//NPT/dump_npt_Si_gap.1000.cfg',
 'GAP18_energy': -162722.24705797626,
 'GAP18_virial': array([[-27.96301042,  -1.9112144 ,  -7.97172699],
        [ -1.9112144 , -42.25210403, -14.6004888 ],
        [ -7.97172699, -14.6004888 , -75.75245511]])}

In [49]:
label_db(GAP18_PQ.df['Configs'].tolist(), GAP_18_pot, 'GAP18')

In [35]:
GAP_18_pot.calculate(GAP_18_db[200], properties=['energies', 'forces', 'virial'])

In [36]:
GAP_18_pot.extra_results['config']['virial']

array([[ 7.12239936e+01, -3.53353646e-01, -6.43044093e-02],
       [-3.53353646e-01,  6.76416908e+01, -6.86723415e-02],
       [-6.43044093e-02, -6.86723415e-02,  6.37228947e+01]])

In [37]:
GAP_18_pot.results['stress']

array([-8.05836337e-02, -7.65305757e-02, -7.20968054e-02,  7.76966657e-05,
        7.27547376e-05,  3.99788321e-04])

In [42]:
mtp_stress_test.calculate(GAP_18_db[200], properties=['forces', 'stress'])

In [43]:
mtp_stress_test.results['stress']

array([[ 6.066771e+01, -2.156200e-01, -7.920000e-03],
       [-2.156200e-01,  5.668705e+01, -7.620000e-03],
       [-7.920000e-03, -7.620000e-03,  4.340911e+01]])

In [8]:
GAP_18_db[0].info

{'config_type': 'isolated_atom',
 'gap_energy': -157.7272532,
 'gap_virial': array([0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'dft_energy': -158.54496821,
 'cutoff': 5.5,
 'nneightol': 1.2,
 'energy': -158.54496700404255,
 'stress': array([-0., -0., -0., -0., -0., -0.]),
 'gap18_energy': -158.54496700404255,
 'gap18_virial': array([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])}

In [65]:
with open('PQ_GAP18labelled_train.cfg', 'w') as f:
    write_cfg_db(f, GAP18_PQ.df['Configs'].tolist(), energy_name='GAP18_energy', force_name='GAP18_force', virial_name='GAP18_virial')

In [ ]:
cfg1 = list(read_cfg_db('GAP_18_db_GAPlabelled.cfg'))

In [9]:
cfg2 = list(read_cfg_db('PQ_GAP18labelled_train.cfg',
                        energy_label='gap18_energy', force_label='gap18_force', stress_label='gap18_virial'))

In [11]:
cfg3 = cfg2 + GAP_18_db

In [14]:
with open('PQ_GAP18_GAP18labelled_train.cfg', 'w') as f:
    write_cfg_db(f, cfg3, 
                 energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_virial')